In [4]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [5]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size=64
epochs=100
latent_dim=256
num_sample=10000


data_path="fra.txt"

In [6]:
#we are reading the input and output data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

with open(data_path,'r',encoding='utf-8') as f:
  lines=f.read().split("\n")

for line in lines[:min(num_sample,len(lines)-1)]:
  input_text,target_text,_=line.split("\t")
  target_text='\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


In [7]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_token=len(input_characters)
num_decoder_token=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [8]:
print("Number of sample",len(input_texts))
print("Number of unique input tokens",num_encoder_token)
print("Number of unique output token",num_decoder_token)
print("Max sequence length of input",max_encoder_seq_length)
print("Max sequence length of output",max_decoder_seq_length)

Number of sample 10000
Number of unique input tokens 71
Number of unique output token 92
Max sequence length of input 16
Max sequence length of output 59


In [9]:
input_token_index=dict(
    [(char,i) for i,char in enumerate(input_characters)])

target_token_index=dict(
    [(char,i) for i,char in enumerate(target_characters)])



In [10]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_token),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_token),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_token),dtype='float32')

In [13]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1

  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]]=1
  
  if t>0:
    decoder_target_data[i,t-1,target_token_index[char]]=1
  decoder_input_data[i,t+1:,target_token_index[' ']]=1
  decoder_input_data[i,t,target_token_index[' ']]=1


In [14]:
encoder_input_data[0].shape

(16, 71)

In [17]:
encoder_inputs=Input(shape=(None,num_encoder_token))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_state=[state_h,state_c]

In [23]:
decoder_inputs=Input(shape=(None,num_decoder_token))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_state)
decoder_dense=Dense(num_decoder_token,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [27]:
model=Model([encoder_inputs,decoder_inputs],decoder_inputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_input_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 1s 4ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 2/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 3/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 4/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 5/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 6/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 7/100
125/125 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 0.0235 - val_accuracy: 1.0000